# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [21]:
import random
from itertools import permutations
from copy import copy
import numpy as np
from tqdm import tqdm
import time
from tqdm import tqdm

In [2]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [54]:
def getCutCapacity(graph, cut):
    ans = 0
    for edge in graph[1]:
        if edge[0] in cut[0] and edge[1] in cut[1]:
            ans += 1
            continue
        if edge[1] in cut[0] and edge[0] in cut[1]:
            ans += 1
            continue
    return ans

def getRandomCut(graph, alpha):
    cut = (set(graph[0]), set())
    while len(cut[0]) / len(graph[0]) > alpha:
        vertex = random.sample(cut[0], 1)[0]
        cut[0].remove(vertex)
        cut[1].add(vertex)
    return cut

def getBestInKLocal(graph, cut, k):
    test = copy(cut)
    best = copy(cut)
    best_score = getCutCapacity(graph, cut)
    better = 1
    for first in permutations(cut[0], k):
        for second in permutations(cut[1], k):
            fs = set(first) | set(second)
            for i in permutations(fs, k):
                fadd = set(i)
                frem = copy(fs).difference(fadd)
                test[0].union(fadd).difference(frem)
                test[1].union(frem).difference(fadd)
                now_score = getCutCapacity(graph, test)
                if now_score < best_score:
                    best_score = now_score
                    best = copy(test)
                    better -= 1
                    if better <= 0:
                        return best
    return best

def basic_local_search(graph, cut):
    k = 1
    now = getCutCapacity(graph, cut)
    while True:
        cut = getBestInKLocal(graph, cut, k)
        new_score = getCutCapacity(graph, cut)
        if new_score >= now:
            break
        now = new_score
    return cut

def randomKJump(graph, cut, k):
    pss = 1
    test = copy(cut)
    for first in permutations(cut[0], k):
        for second in permutations(cut[1], k):
            pss -= 1
            if pss > 0:
                continue
            fs = set(first) | set(second)
            for i in permutations(fs, k):
                fadd = set(i)
                frem = copy(fs).difference(fadd)
                test[0].union(fadd).difference(frem)
                test[1].union(frem).difference(fadd)
                now_score = getCutCapacity(graph, test)
                return test
    return test

In [56]:
def variable_depth_local_search(graph):
    starting_point = set(range(1, len(graph[0]) // 2 + 1))
    cut = (starting_point, graph[0].difference(starting_point))
    basic_local_search(graph, cut)
    iterations = 0
    for i in tqdm(range(iterations)):
        cut = randomKJump(graph, cut, 3)
        basic_local_search(graph, cut)
    return cut[0]

In [57]:
def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]).difference(partition_part)
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

Для сравнения, это запуск на разбиении равном начальному

In [10]:
run_all()

Solving instance add20.graph… done in 0.026 seconds with quality 11070
Solving instance cti.graph… done in 0.13 seconds with quality 94052
Solving instance t60k.graph… done in 0.26 seconds with quality 178236
Solving instance m14b.graph… done in 5.4 seconds with quality 2888974


In [58]:
run_all()

Solving instance add20.graph…

KeyboardInterrupt: 

## Выводы
Хм, что-то в начальной версии оно просто безумно долго работает, при любых параметрах.